In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE  {catalog_name}.udp_wcm_gold_vcm_dwh.f_saleout_bill_header AS 
WITH
lty_bill AS (
  SELECT
    invoice AS receipt_no,
    b.mobile_number AS member_card_no
  FROM  {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_transactions a
  LEFT JOIN (
    SELECT
      e.id AS customer_id,
      {catalog_name}.default.FN_CLEAN_MOBILE_NO(a.merchant_customer_id) AS mobile_number
    FROM {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_customer_merchant_rels a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time b
      ON DATE(FROM_UTC_TIMESTAMP(TRY_TO_TIMESTAMP(a.create_date), 'UTC+7')) = b.calday
    LEFT JOIN  {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_loyalty_customers e 
      ON a.csn = e.csn
    WHERE
      a.merchant_id = 'WIN'
      AND DATE(FROM_UTC_TIMESTAMP(TRY_TO_TIMESTAMP(a.create_date), '+07:00')) <= '2023-12-05'
      AND e.is_deleted <> 'true'
      AND a.is_deleted <> 'true'
    ) b ON a.customer_id = b.customer_id
  WHERE pool_id = 'WIN' AND merchant_id = 'VCM'
),
winx_bill_props AS (
  SELECT DISTINCT receipt_no, prop_value
  FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_props
  WHERE prop_name = 'winxmembercode'
)
SELECT
  a.receipt_no,
  pos_no,
  store_id,
  calday,
  caltime,
  IFNULL(a.member_card_no,b.member_card_no) mobile_number,
  UNHEX(MD5(IFNULL(a.member_card_no,b.member_card_no))) mobile_number_enc,
  vinid_csn,
  header_ref_01 ref_01_ref_bill,
  header_ref_02 ref_02_sale_channel,
  header_ref_03 ref_03_origin_trx_num,
  header_ref_04 ref_04,
  header_ref_05 ref_05,
  c.prop_value winx_member_code
FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_header a
LEFT JOIN lty_bill b ON a.receipt_no = b.receipt_no
LEFT JOIN winx_bill_props c ON a.receipt_no = c.receipt_no
""")
